# MIT BIH heart beat classification model for heartbeats


In [ ]:
pip install neurokit.py

In [1]:
import numpy as np
import pandas as pd
import csv
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import tree
%run Functions.ipynb
%run FeaturesFunctions.ipynb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

loaded functions


First we need to create a training file with all of the annotated files in order
File name: 1 csv for feature extraction

In [26]:

AnnotatedFileNames=[*range(100,125)]+[*range(200,235)]
remove=[110,113,120,204,206,211,216,218,224,225,226,227,229]
for i in remove:
    AnnotatedFileNames.remove(i)
Training = pd.DataFrame(columns = ['File','Result', 'Sample#'] )

AnnotatedFileNames=AnnotatedFileNames[:5]
for i in AnnotatedFileNames:
    temp= pd.DataFrame(columns = Training.columns)

    FileName ="mitbih_database/" + str(i)+'annotations.txt'
    annotations = mlbeats.df_from_txt(FileName)
    temp['Sample#']=annotations['Sample#']
    temp['Result']=annotations['Result']
    temp['File']= i
    
    Training = pd.concat([Training, temp], axis=0)
    

In [27]:
Training.head()
print(len(Training))

10742


Adding neurokit data and aux data to training file.ipynbThen using the nerokit library we can add feature locations to set features in the data, Filename: Adding neurokit data and aux data to training file.ipynb

In [8]:
testFeatures=pd.DataFrame()
for file in AnnotatedFileNames:
    DATA= pd.DataFrame(np.nan, index=range(1), columns=testFeatures.columns)
    testFeatures=pd.concat([testFeatures,DATA])
    testFeatures=pd.concat([testFeatures,Bryson.NerokitExtraction(file)])


extracting File 100
extraction complete
extracting File 101
extraction complete
extracting File 102
extraction complete
extracting File 103
extraction complete
extracting File 104
extraction complete


In [28]:
#NewTraining=pd.concat([NewTraining,NewFeatures],axis=1)
Training = Training.reset_index(drop=True)
testFeatures=testFeatures.reset_index(drop=True)
Training=pd.concat([Training,testFeatures],axis=1)
Training = Training.reset_index()
Training.head()

,index,File,Result,Sample#,beat_number,ECG_R_Peaks,ECG_Q_Peaks,ECG_S_Peaks,ECG_P_Peaks,ECG_P_Onsets,ECG_P_Offsets,ECG_T_Peaks,ECG_T_Onsets,ECG_T_Offsets,ECG_R_Onsets,ECG_R_Offsets
0,0,100,+,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100,N,77,1,77.0,55.0,174.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100,N,370,2,370.0,347.0,467.0,311.0,283.0,344.0,NaN,NaN,NaN,314.0,NaN
3,3,100,N,662,3,662.0,641.0,697.0,606.0,578.0,638.0,NaN,NaN,NaN,612.0,NaN
4,4,100,N,946,4,946.0,926.0,971.0,886.0,858.0,923.0,NaN,NaN,NaN,892.0,NaN


In [29]:

Training = Training.drop(["ECG_T_Peaks","ECG_T_Onsets","ECG_T_Offsets","ECG_R_Offsets"],axis='columns')
print(len(Training))
Training.dropna(inplace=True)
print(len(Training))

10742
7188


In [30]:
Interval = pd.DataFrame(columns=["File","Abnormal","Result","R-R Interval","R Height","R_Onset-Rpeak","Q-Q Interval","Q Height","P_Onset-P_Offset","P Height","P-P Interval","P_Onset-Ppeak"])
RRLIST=np.array([])
QQLIST=np.array([])
R_Height=np.array([])
Q_Height=np.array([]) 
P_Height=np.array([])
Ron_RP=np.array([])
Pon_PP=np.array([])
Pon_Poff=np.array([])
PPLIST=np.array([])
Abnormal=np.array([])
Result=np.array([])
FILES=np.array([])
y=2
FileNumber=Training["File"][y]
RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")



In [31]:


for i in Training["index"][:-1]:
    #FileNumber=Training["File"][i]
    #df=pd.DataFrame(columns=Interval.columns)
    #RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")
    
    if (i-y>1):
        y=i
        continue
    if Training["Result"][i]=='N':
        Abnormal=np.append(Abnormal,0)
    else:
        Abnormal=np.append(Abnormal,1)
    if (Training["File"][i] != FileNumber):
        print("Reading File",FileNumber)
        FileNumber=Training["File"][i]
        RAWFile= pd.read_csv("mitbih_database/"+str(FileNumber)+".csv")
    #if (i%100 ==0):
        #print(i,":",time.time()-start)

    RRLIST=np.append(RRLIST,Training["ECG_R_Peaks"][i]-Training["ECG_R_Peaks"][y])
    QQLIST=np.append(QQLIST,Training["ECG_Q_Peaks"][i]-Training["ECG_Q_Peaks"][y])
    PPLIST=np.append(PPLIST,Training["ECG_P_Peaks"][i]-Training["ECG_P_Peaks"][y])
    R_Height=np.append(R_Height,RAWFile[RAWFile.columns[1]][Training["ECG_R_Peaks"][i]])#
    Q_Height=np.append(Q_Height,RAWFile[RAWFile.columns[1]][Training["ECG_Q_Peaks"][i]])#
    P_Height=np.append(P_Height,RAWFile[RAWFile.columns[1]][Training["ECG_P_Peaks"][i]])#
    Ron_RP=np.append(Ron_RP,Training["ECG_R_Peaks"][i]-Training["ECG_R_Onsets"][i])
    Pon_PP=np.append(Pon_PP,Training["ECG_P_Peaks"][i]-Training["ECG_P_Onsets"][i])
    Pon_Poff=np.append(Pon_Poff,Training["ECG_P_Offsets"][i]-Training["ECG_P_Onsets"][i])
    Result=np.append(Result,Training["Result"][i])
    FILES=np.append(FILES,FileNumber)
    
    y=i

Interval["R-R Interval"]=RRLIST
Interval["Q-Q Interval"]=QQLIST
Interval["P-P Interval"]=PPLIST
Interval["R_Onset-Rpeak"]=Ron_RP
Interval["P_Onset-Ppeak"]=Pon_PP
Interval["P_Onset-P_Offset"]=Pon_Poff
Interval["R Height"]=R_Height#
Interval["Q Height"]=Q_Height#
Interval["P Height"]=P_Height#
Interval["File"]=FILES
Interval["Abnormal"]=Abnormal
Interval["Result"]=Result

print("Length of DataFrame",len(Interval))
Interval.head(10)





Reading File 100
Reading File 101
Reading File 102
Reading File 103
Length of DataFrame 6585


,File,Abnormal,Result,R-R Interval,R Height,R_Onset-Rpeak,Q-Q Interval,Q Height,P_Onset-P_Offset,P Height,P-P Interval,P_Onset-Ppeak
0,100.0,0.0,N,0.0,1212.0,56.0,0.0,953.0,61.0,978.0,0.0,28.0
1,100.0,0.0,N,292.0,1201.0,50.0,294.0,947.0,60.0,976.0,295.0,28.0
2,100.0,0.0,N,284.0,1186.0,54.0,285.0,946.0,65.0,971.0,280.0,28.0
3,100.0,0.0,N,285.0,1188.0,59.0,284.0,948.0,66.0,973.0,283.0,29.0
4,100.0,0.0,N,284.0,1201.0,56.0,283.0,952.0,64.0,977.0,285.0,28.0
5,100.0,0.0,N,294.0,1213.0,58.0,294.0,956.0,64.0,983.0,293.0,27.0
6,100.0,1.0,A,235.0,1193.0,61.0,235.0,954.0,50.0,981.0,233.0,22.0
7,100.0,0.0,N,358.0,1193.0,55.0,359.0,943.0,64.0,973.0,361.0,28.0
8,100.0,0.0,N,304.0,1202.0,57.0,303.0,948.0,64.0,972.0,304.0,28.0
9,100.0,0.0,N,292.0,1209.0,52.0,292.0,955.0,61.0,978.0,296.0,29.0


In [32]:
Interval.dropna(inplace=True)
Interval.head()

#creatin
model1 = tree.DecisionTreeClassifier()

inputs = Interval.drop(["Result","Abnormal"],axis='columns')
Target = Interval[["Abnormal"]]



x_train1, x_test1, y_train1, y_test1 = train_test_split(inputs, Target,test_size=0.3)

model1.fit(X=x_train1,y=y_train1)
print(model1.score(x_test1,y_test1),"Regular")
print(roc_auc_score(y_test1, model1.predict(x_test1)),"AUC1\n")





0.993421052631579 Regular
0.9699257192248123 AUC1

